# Improving model performance with xfeat, RAPIDS and Optuna


## Introduction

Feature Engineering is the processing of transforming raw data into features that can represent the underlying patterns of the data better. They can help boost the accuracy by a great deal and improve the ability of the model to generalise on unseen data. Every data scientist knows the importance feature engineering. Spending some time thinking about how best to apply and combine the available features can be very meaningful. 

Hyper parameter Optimisation is another such process which can help complement a good model by tuning it's hyperparameters, which can have a tremendous impact on the accuracy of the model. The time and resources required for these processes are generally the reason they're overlooked. 

With xfeat, RAPIDS and Optuna - we aim to bridge these gaps and elevate the performance. 

## What is Optuna?
[Optuna](https://github.com/optuna/optuna) s a lightweight framework for automatic hyperparameter optimization. It provides a define-by-run API, which makes it easy to adapt to any already existing code that we have and enables high modularity and the flexibility to construct hyperparameter spaces dynamically. By simply wrapping the objective function with Optuna can help perform a parallel-distributed HPO search over a search space. As we'll see in this notebook.

## What is xfeat?
[xfeat](https://github.com/pfnet-research/xfeat) is a feature engineering & exploration library using GPUs and Optuna. It provides a scikit-learn-like API for feature engineering with support for pandas and cuDF dataframes and cuPy arrays. 

## What is MLflow?
[MLflow](https://mlflow.org/https://mlflow.org/) MLflow is an, "open source platform to manage the ML lifecycle, including experimentation, reproducibility, deployment, and a central model registry".

## What is RAPIDS?
[RAPIDS](https://rapids.ai/about.html) framework  provides a library suite that can execute end-to-end data science pipelines entirely on GPUs.  The libraries in the framework include [cuDF](https://github.com/rapidsai/cudf) - a GPU Dataframe with pandas-like API, [cuML](https://github.com/rapidsai/cuml) - implement machine learning algorithms that provide a scikit-learn-like API and many more. You can learn more [here](https://github.com/rapidsai).

In this notebook, we'll show how one can use these tools together to develop and improve a machine learning model. We'll use Airlines dataset (20M rows) to predict if a flight will be delayed or not. We'll explore how to use Optuna with RAPIDS and the speedups that we can achieve with the integration of these, and to see the improvements with GPU speedups, we have included a pandas version to run on CPU. A table summarizing the results is available at the end of the notebook.

In [1]:
import time
import json
import requests
import logging

import numpy as np

import cupy
import cudf
import cuml
from cuml import LogisticRegression
from cuml.metrics import accuracy_score
from cuml.preprocessing.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient

from mlflow.models.signature import infer_signature

import optuna
from optuna.integration.mlflow import MLflowCallback
from optuna.study import StudyDirection
from optuna.trial import TrialState
from optuna import type_checking

import xfeat
from xfeat.pipeline import Pipeline
from xfeat.num_encoder import SelectNumerical
from xfeat.selector import ChiSquareKBest
from xfeat.optuna_selector import KBestThresholdExplorer, GroupCombinationExplorer
from functools import partial

import sklearn
from sklearn.model_selection import KFold
import pandas as pd

from xfeat import ArithmeticCombinations, Pipeline, SelectNumerical, LabelEncoder, SelectCategorical, TargetEncoder

In [2]:

import time
from contextlib import contextmanager
# Helping time blocks of code
@contextmanager
def timed(txt):
    t0 = time.time()
    yield
    t1 = time.time()
    print("%32s time:  %8.5f" % (txt, t1 - t0))

### MLflow Configuration

For tracking the hyperparameter optimisation expriments, we will use MLFlow. In the next cell, the required variables are set up along with the callback class `RAPIDSMLflowCallback` that we will pass to Optuna for tracking.

In [3]:
MLFLOW_TRACKING_URI='sqlite:////tmp/mlflow-db.sqlite'
MLFLOW_MODEL_ID = "rapids-optuna-airline"

def get_latest_mlflow_model(tracking_uri, model_id):
    client = MlflowClient(tracking_uri=tracking_uri, registry_uri=tracking_uri)
    model = client.get_registered_model(model_id)
    latest_model = model.latest_versions[0]

    return f"MLFLOW_TRACKING_URI={tracking_uri} mlflow models serve --no-conda -m models:/{model_id}/{latest_model.version} -p 56767"

## Custom callback, for additional flexibility, based on MLflowCallback
class RAPIDSMLflowCallback(object):
    def __init__(self, tracking_uri: str = "sqlite:////tmp/mlflow-db.sqlite",
                 experiment_name: str = "RAPIDS-Optuna",
                 metric_name="value"):
        self._tracking_uri = tracking_uri
        self._experiment_name = experiment_name
        self._metric_name = metric_name
        
    def __call__(self, study, trial):
        if (self._tracking_uri is not None):
            mlflow.set_tracking_uri(self._tracking_uri)
        
        eid = mlflow.set_experiment(self._experiment_name)
        with mlflow.start_run(run_name=f"Trial: {trial.number}", experiment_id=eid, nested=True):
            trial_value = trial.value if trial.value is not None else float("nan")
            mlflow.log_metric(self._metric_name, trial_value)
            
            mlflow.log_params(trial.params)

            tags = {}
            tags["number"] = str(trial.number)
            tags["datetime_start"] = str(trial.datetime_start)
            tags["datetime_complete"] = str(trial.datetime_complete)
            tags['RAPIDS cuDF Version'] = str(cudf.__version__)
            tags['RAPIDS cuML Version'] = str(cuml.__version__)
            tags['SKlearn Version'] = str(sklearn.__version__)

            trial_state = trial.state
            if (isinstance(trial_state, TrialState)):
                tags['state'] = str(trial_state).split('.')[-1]
            
            # Set direction and convert it to str and remove the common prefix.
            study_direction = study.direction
            if isinstance(study_direction, StudyDirection):
                tags["direction"] = str(study_direction).split(".")[-1]

            tags.update(trial.user_attrs)
            distributions = {
                (k + "_distribution"): str(v) for (k, v) in trial.distributions.items()
            }
            tags.update(distributions)

            # This is a temporary fix on Optuna side. It avoids an error with user
            # attributes that are too long. It should be fixed on MLflow side later.
            # When it is fixed on MLflow side this codeblock can be removed.
            # see https://github.com/optuna/optuna/issues/1340
            # see https://github.com/mlflow/mlflow/issues/2931
            max_mlflow_tag_length = 5000
            for key, value in tags.items():
                value = str(value)  # make sure it is a string
                if len(value) > max_mlflow_tag_length:
                    tags[key] = textwrap.shorten(value, max_mlflow_tag_length)

            mlflow.set_tags(tags) 

### Feature Engineering

We'll be using the following functions to perform a few feature engineering tasks on the data. The `feature_engineering` function is called on the dataframe `df`, in this function we perform a simple Arithmetic Combinations on the numerical columns that adds two columns to create a new one. We specify the `operator` and `r` - r is used to indicate how many columns need to be combined.

Then we call `categorical_encoding` which converts the categorical columns to numerical ones and then performs `target_encoding`. Target Encoding replaces the value with the target mean. This is helpful in classification problem to boost the model accuracy. Find more resources at the end of the notebook.

You'll also notice we use `Pipeline` from xfeat to combine two or more feature engineering tasks together. This is useful to concatenate encoders sequentially.

Read more about Feature Encoding and Pipelining with xfeat [here](https://github.com/pfnet-research/xfeat/blob/master/_docs/feature_encoding.md).

In [4]:
def feature_engineering(df):
    """
    Perform feature engineering and return a new df with engineered features
    """
    if isinstance(df, cudf.DataFrame):
        df_train, df_test, y_train, y_test = train_test_split(df, "ArrDelayBinary",random_state=np.random.seed(0), shuffle=True)
    else:
        from sklearn.model_selection import train_test_split as sk_split
        df_train, df_test, y_train, y_test = sk_split(df[df.columns.difference(["ArrDelayBinary"])], df["ArrDelayBinary"],random_state=np.random.seed(0), shuffle=True)

    # Xfeat's internal fold mechanism creates RangeIndex references, so we need to do an index reset on our data frames.
    df_train = df_train.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    # Need to do this to ensure we are appropriately assigning the split values
    df_train["ArrDelayBinary"] = y_train
    df_test["ArrDelayBinary"] = y_test
    
    # combine into one pipeline
    encoder = Pipeline([
                        LabelEncoder(output_suffix=""),
                        TargetEncoder(target_col="ArrDelayBinary", output_suffix=""),
                        ArithmeticCombinations(exclude_cols=["ArrDelayBinary"],
                                               drop_origin=False,
                                               operator="+",
                                               r=2,
                                               output_suffix="_plus")
                    ])
    df_train = encoder.fit_transform(df_train)
    df_test = encoder.transform(df_test)
    if isinstance(df, cudf.DataFrame):
        df = cudf.concat([df_train, df_test], sort=False)
    else:
        df = pd.concat([df_train, df_test], sort=False)
    return df

### Feature Selection and Hyper parameter Optimisation

Now that we have some new features, how do we know they are relevant for the task or represent anything meaningful? We use the feature selection process to do this. This helps in selection of a subset of features that are  most informative. This helps in simplifying the problem and ensures that we aren't overloading the system with unimportant features. Optuna provides a way to choose a `selector` which accepts a `Pipeline` object from xfeat. You can see in the `feature_selection` function we define a Pipeline that takes in an Explorer and a Selection Algorithm (`ChiSquareKBest`). We pass this to an Optuna Study object, along with an Objective function

### Objective Function
The objective function will be the one we optimize in Optuna Study. Objective funciton tries out different values for the parameters that we are tuning and saving the results in `study.trials_dataframes()`.

Let's define the objective function for this HPO task by making use of the `train_and_eval()`. You can see that we simply choose a value for the parameters and call the `train_and_eval` method, making Optuna very easy to use in an existing workflow.

The objective remains constant over different samplers, which are built-in options in Optuna to enable the selection of different sampling algorithms that optuna provides. Some of the available ones include - GridSampler, RandomSampler, TPESampler, etc. We'll use TPESampler for this demo, but feel free to try different samplers to notice the chnages in performance.


### HPO Trials and Study
Optuna uses [study](https://optuna.readthedocs.io/en/stable/reference/study.html) and [trials](https://optuna.readthedocs.io/en/stable/reference/trial.html) to keep track of the HPO experiments. Put simply, a trial is a single call of the objective function while a set of trials make up a study. We will pick the optimal performing trial from a study to get the best parameters that were used in that run.

In [5]:
def train_and_eval(df, penalty='l2', C=1.0, l1_ratio="None", fit_intercept=True, selector=None, return_model=False):
    # Splitting data and prepping for selector fit
    if isinstance(df, cudf.DataFrame):
        X_train,  X_test, y_train, y_test = train_test_split(df, "ArrDelayBinary",random_state=np.random.seed(0), shuffle=True)
    else:
        from sklearn.model_selection import train_test_split as sk_split
        X_train,  X_test, y_train, y_test = sk_split(df[df.columns.difference(["ArrDelayBinary"])], df["ArrDelayBinary"],random_state=np.random.seed(0), shuffle=True)
    # Xfeat's internal fold mechanism creates RangeIndex references, so we need to do an index reset on our data frames.

    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    if selector:
        with timed("Feature selector"):
            # For the selector, the label also needs to be in the DF
            X_train["ArrDelayBinary"] = y_train
            X_test["ArrDelayBinary"] = y_test

            X_train = selector.fit_transform(X_train)
            X_test = selector.transform(X_test)
    
    # Train and get accuracy
    if isinstance(df, cudf.DataFrame):
        classifier = LogisticRegression(penalty=penalty,
                                        C=C,
                                        l1_ratio=l1_ratio,
                                        fit_intercept=fit_intercept,
                                        max_iter=10000)
        with timed("Training and evaluation"):
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict_proba(X_test.values)[:, 1]
            score = accuracy_score(y_test, y_pred)
    else:
        from sklearn.linear_model import LogisticRegression as sk_log
        classifier = sk_log(solver='saga',
                            penalty=penalty,
                                        C=C,
                                        l1_ratio=l1_ratio,
                                        fit_intercept=fit_intercept,
                                        max_iter=10000)
        with timed("Training and evaluation"):
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_test)
            from sklearn.metrics import accuracy_score as sk_acc
            score = sk_acc(y_test.astype('float64'), y_pred)
    
    
    if (return_model):
        if isinstance(X_test, cudf.DataFrame):
            return score, classifier, infer_signature(X_test.to_pandas(), cupy.asnumpy(y_pred))
        else:
            return score, classifier, infer_signature(X_test, y_pred)
    
    return score

def objective(df, selector, trial):
    """
    Performs the training and evaluation of the set of parameters and subset of features using selector.
    """
    selector.set_trial(trial)
    
    # Select Params
    C = trial.suggest_uniform("C", 0 , 7.0)
    penalty = trial.suggest_categorical("penalty", ['l1', 'none', 'l2'])
    l1_ratio = trial.suggest_uniform("l1_ratio", 0 , 1.0)
    fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
    
    score = train_and_eval(df,
                           penalty=penalty,
                           C=C,
                           l1_ratio=l1_ratio,
                           fit_intercept=fit_intercept,
                           selector=selector)
    return score

def feature_selection(df, experiment_name):
    """
    Defines the Pipeline and performs the optuna opt
    """
    artifact_path = "rapids-optuna-airline"
    selector = Pipeline(
        [
            SelectNumerical(),
            KBestThresholdExplorer(ChiSquareKBest(target_col="ArrDelayBinary")),
        ]
    )

    mlfcb = RAPIDSMLflowCallback(
        tracking_uri=MLFLOW_TRACKING_URI,
        experiment_name=experiment_name,
        metric_name='accuracy')
    
    study = optuna.create_study(direction="maximize")
    

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=f"Optuna-HPO:{study.study_name}"):
        study.optimize(partial(objective, df, selector), n_trials=N_TRIALS, callbacks=[mlfcb])
        
        selector.from_trial(study.best_trial)
        selected_cols = selector.get_selected_cols()
        df_select = df[selected_cols]
        df_select['ArrDelayBinary'] = df['ArrDelayBinary']
        
        params = study.best_params
        score, classifier, signature = train_and_eval(df_select,
                      C=params['C'],
                      penalty=params['penalty'],
                      l1_ratio=params['l1_ratio'],
                      fit_intercept=params['fit_intercept'],
                      return_model=True)
        
        with mlflow.start_run(run_name='Final Classifier', nested=True):
            mlflow.log_metric('accuracy', score)
            mlflow.log_params(params)
            mlflow.sklearn.log_model(classifier,
                                 signature=signature,
                                 artifact_path=artifact_path,
                                 registered_model_name="rapids-optuna-airline",
                                 conda_env='conda/conda.yaml')

    return study, df_select.reset_index(drop=True)           

### Set Experiment Variables

Change the `INPUT_FILE` to correspond to the path in your local system and select the number of rows and trials to run the experiment for. 

NOTE: It is not recommended to run the CPU version for more than 100,000 rows. To avoid this, we set a glad `cpu_run` based on `N_ROWS`

In [6]:
INPUT_FILE = "/home/data/airline_data/air_par.parquet"

N_ROWS = 10000000
N_TRIALS = 100

cpu_run = N_ROWS <= 100000

In [7]:
if cpu_run:
    with timed("Complete Pandas run"):
        df_pandas = pd.read_parquet(INPUT_FILE)[:N_ROWS]
        print("Default scikit-learn ", train_and_eval(df_pandas))
        with timed("Pandas FE"):
            df_pandas_fe = feature_engineering(df_pandas)
            df_pandas_fe["ArrDelayBinary"] = df_pandas_fe["ArrDelayBinary"].astype('float64')
            score = train_and_eval(df_pandas_fe)
        print("After feature eng: ", score)

        # Disable Alembic driver, used by MLflow, from logging INFO messages to the command line.
        with timed("FS + Optuna PANDAS"):
            logging.getLogger('alembic').setLevel(logging.CRITICAL)
            study, df_select = feature_selection(df_pandas_fe, experiment_name='Optuna Testing')

## GPU Run

Now, let's run the RAPIDS version by first reading in the data as cudf DataFrames.

In [8]:
with timed("Complete RAPIDS workflow"):
    df_ = cudf.read_parquet(INPUT_FILE)[:N_ROWS]

    df_ = df_.drop(["ActualElapsedTime"], axis=1)
    # Can't handle nagative values, yet
    # indices = df_.loc[df_["ActualElapsedTime"] < 0].index
    # df_.loc[indices, "ActualElapsedTime"] = -1 * df_.loc[indices, "ActualElapsedTime"]
    # cuML can't handle object types
    # df_["ArrDelayBinary"] = df_["ArrDelayBinary"].astype('int32')
    with timed("Default"):
        print("Default performance: ", train_and_eval(df_))
    # We cast to objects for categorical  and target encoding
    df_["UniqueCarrier"] = df_["UniqueCarrier"].astype("object")
    df_["Origin"] = df_["Origin"].astype("object")
    df_["Dest"] = df_["Dest"].astype("object")
    with timed("FE"):
        df_feature_eng = feature_engineering(df_)
        df_feature_eng["ArrDelayBinary"] = df_feature_eng["ArrDelayBinary"].astype('float64')
        score = train_and_eval(df_feature_eng)
        print("After feature eng: ", score)
    # Disable Alembic driver, used by MLflow, from logging INFO messages to the command line.
    with timed("FS + Optuna"):
        logging.getLogger('alembic').setLevel(logging.CRITICAL)
        study, df_select = feature_selection(df_feature_eng, experiment_name='Optuna Testing')

         Training and evaluation time:   5.66595
Default performance:  0.8126136064529419
                         Default time:   5.86765
         Training and evaluation time:   0.72254
After feature eng:  0.7998239994049072
                              FE time:  51.24827
                Feature selector time:   6.52997
         Training and evaluation time:   0.31214


[I 2020-08-26 19:11:17,628] Trial 0 finished with value: 0.8117247819900513 and parameters: {'C': 5.015092132364396, 'penalty': 'none', 'l1_ratio': 0.57935976991689, 'fit_intercept': True, 'KBestThresholdExplorer.k': 6.0}. Best is trial 0 with value: 0.8117247819900513.


                Feature selector time:   6.60180
         Training and evaluation time:   1.89756


[I 2020-08-26 19:11:26,915] Trial 1 finished with value: 0.8121140003204346 and parameters: {'C': 3.3602709849833055, 'penalty': 'l2', 'l1_ratio': 0.8786448416474414, 'fit_intercept': True, 'KBestThresholdExplorer.k': 63.0}. Best is trial 1 with value: 0.8121140003204346.


                Feature selector time:   6.56542
         Training and evaluation time:  23.41429


...

                Feature selector time:   6.40883
         Training and evaluation time:   2.05524


[I 2020-08-26 19:27:36,308] Trial 99 finished with value: 0.8118224143981934 and parameters: {'C': 2.3648534254827274, 'penalty': 'none', 'l1_ratio': 0.15904578440667652, 'fit_intercept': True, 'KBestThresholdExplorer.k': 64.0}. Best is trial 85 with value: 0.8122171759605408.
Optuna selector is not last component.


         Training and evaluation time:   0.90711


Registered model 'rapids-optuna-airline' already exists. Creating a new version of this model...
Created version '17' of model 'rapids-optuna-airline'.


                     FS + Optuna time:  988.27946
        Complete RAPIDS workflow time:  1047.02740


In [9]:
df_select["ArrDelayBinary"] = df_["ArrDelayBinary"].astype('float64')

In [10]:
params = study.best_params
score, classifier, signature = train_and_eval(df_select,
                      C=params['C'],
                      penalty=params['penalty'],
                      l1_ratio=params['l1_ratio'],
                      fit_intercept=params['fit_intercept'],
                      return_model=True)

print("After feature selection and paramter tuning: ", score)


         Training and evaluation time:   0.38069
After feature selection and paramter tuning:  0.8126248121261597


## Performance Summarization

We see that GPU runs are faster for 100,000 rows and we are able to obtain speedups of `TBD`. For more performance improvements, you are encouraged to increased the number of rows and/or number of trials. This will result in a big boost in accuracy. Keep in mind, you do not want to run the experiment on CPU with a larger number of rows, as this will result in the kernel crashing.

|Number of rows| Trials| CPU | GPU|
|-|-|-|-|
|100,000|10|250.09533|14.79310|
|1,000,000|10|-|31.44453|
|10,000,000|10|-|315.21524|
|10,000,000|100|-|1047.02740|

Now, that we have seen the performance improvement, let's look at how we can retrieve this model with MLFlow.

### Launch our optimized model within the MLflow framework.
Run the code block below to identify the most recently registered model, with the 'rapids-optuna-airline' tag; after identifying the latest model version, run the code below in a separate terminal, and wait for it to fully load your model.

In [11]:
print(f"Run the command below in a terminal, and wait for it to load your model:\n\n  \
      {get_latest_mlflow_model(MLFLOW_TRACKING_URI, MLFLOW_MODEL_ID)}")

Run the command below in a terminal, and wait for it to load your model:

        MLFLOW_TRACKING_URI=sqlite:////tmp/mlflow-db.sqlite mlflow models serve --no-conda -m models:/rapids-optuna-airline/17 -p 56767


You should see output similar to the following:

```shell
2020/07/27 13:59:49 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2020/07/27 13:59:49 INFO mlflow.pyfunc.backend: === Running command 'source /anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-3335621df6011b1847d2555b195418d4496e5ffd 1>&2 && gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2020-07-27 13:59:50 -0600] [23779] [INFO] Starting gunicorn 20.0.4
[2020-07-27 13:59:50 -0600] [23779] [INFO] Listening at: http://127.0.0.1:5000 (23779)
[2020-07-27 13:59:50 -0600] [23779] [INFO] Using worker: sync
[2020-07-27 13:59:50 -0600] [23788] [INFO] Booting worker with pid: 23788
```

In [12]:
host='localhost'
port='56767'

headers = {
    "Content-Type": "application/json",
    "format": "pandas-split"
}

data = { 
    "columns": ["Year", "Month", "DayofMonth", "DayofWeek", "CRSDepTime", "CRSArrTime", "UniqueCarrier",
                "FlightNum", "ActualElapsedTime", "Origin", "Dest", "Distance", "Diverted"],
    "data": [[1987, 10, 1, 4, 1, 556, 0, 190, 247, 202, 162, 1846, 0]]
}

while (True):
    try:
        resp = requests.post(url="http://%s:%s/invocations" % (host, port), data=json.dumps(data), headers=headers)
        print('Classification: %s' % ("ON-Time" if resp.text == "[0.0]" else "LATE"))
        break
    except Exception as e:
        errmsg = "Caught exception attempting to call model endpoint: %s" % e
        print(errmsg)
        print("... Sleeping ...")
        time.sleep(20)

## Additional Resources
[How to Win a DS Kaggle competition](https://www.coursera.org/learn/competitive-data-science)

[Target Encoding and Bayesian Target Encoding](https://towardsdatascience.com/target-encoding-and-bayesian-target-encoding-5c6a6c58ae8c)
